# Bayesian Optimization

We implement a Bayesian optimization for LED configurations that most closely match a target
light spectrum. We will work with a helper class, `SelfDrivingLabDemo` for setting up the
spectrophotometer sensor, measuring data from the sensor,
generating random inputs, and measuring the objective function (i.e. mean absolute
error). From there, we perform an experiment with 100 iterations, visualize the
results, and compare against random search. See
[2.0-random-search.ipynb](2.0-random-search.ipynb) for some links on troubleshooting
installation setup and a linear version of running an optimization campaign without the
`SelfDrivingLabDemo` class.

## Setup

Imports and the main class.

### Imports

In [1]:
%load_ext autoreload
%autoreload 2 # just some IPython magic to recognize changes to installed packages
import pandas as pd
from self_driving_lab_demo.core import SelfDrivingLabDemo

### SelfDrivingLabDemo

We'll instantiate the class and verify some of the functionality described in the random
search tutorial ([`2.0-random-search.ipynb`](2.0-random-search.ipynb)).

#### Instantiation

Now, we instantiate the `SelfDrivingLabDemo` class with `autoload=True` so that records
a target to optimize against. This involves selecting a set of target measurements as the "true" input values (i.e. the input
brightness and RGB values that define the target spectrum) based on a random seed,
setting the LED to those values, and then recording the spectrum intensities.

> Note: Instantiating with autoload=True will light the LED briefly.

In [2]:
sdl = SelfDrivingLabDemo(autoload=True)

#### Functionality

We can do similar things to what was done in `2.0-random-search.ipynb`. For example, getting
random inputs, observing the sensor data, and evaluating the objective function.

In [3]:
[sdl.get_random_inputs(), sdl.get_random_inputs()]

[(0.27088461699458716, 112, 219, 178), (0.032962071760677336, 249, 194, 200)]

In [4]:
sdl.observe_sensor_data(*sdl.get_random_inputs())

(314, 2129, 2196, 1575, 738, 1067, 1691, 1219)

In [5]:
sdl.evaluate(*sdl.get_random_inputs())

{'ch415_violet': 602,
 'ch445_indigo': 3314,
 'ch480_blue': 3909,
 'ch515_cyan': 7842,
 'ch560_green': 1973,
 'ch615_yellow': 6624,
 'ch670_orange': 12736,
 'ch720_red': 1532,
 'mae': 3158.75,
 'rmse': 4783.712731759715}

## Optimization

While there are great numerical tutorials comparing [grid search vs. random search vs.
Bayesian optimization](https://towardsdatascience.com/grid-search-vs-random-search-vs-bayesian-optimization-2e68f57c3c46), here, we'll compare these three search methods in a way that perhaps you've never seen before,
namely a self-driving laboratory demo!

### Setup

We define our optimization task parameters and take care of imports.

### Optimization Task Parameters

We'll use 81 iterations repeated 5 times. The use of 81 iterations instead of something
"cleaner" like 50 or 100 is due to constraints of doing uniform (full-factorial) grid
search. $n^d$ number of points are required for uniform grid search, where $n$ and $d$
represent number of points per dimension (`n_pts_per_dim`) and number of dimensions
(`4`), respectively.

In [6]:
num_iter = 3 ** 4
num_repeats = 5
SEEDS = range(10, 10 + num_repeats)

We also instantiate multiple `SelfDrivingLabDemo` instances, each with their own
unique target spectrum.

In [7]:
sdls = [SelfDrivingLabDemo(autoload=True, target_seed=seed) for seed in SEEDS]

Notice that the target_data is different for each.

In [8]:
pd.DataFrame([sdl.target_data for sdl in sdls], columns=sdl.channel_names)

,ch415_violet,ch445_indigo,ch480_blue,ch515_cyan,ch560_green,ch615_yellow,ch670_orange,ch720_red
0,725,3680,7084,21221,3924,3668,6416,1724
1,278,306,572,2162,853,1092,1731,1133
2,332,1013,1146,1558,744,2614,4775,1208
3,952,5662,7941,19392,3946,10356,20214,1977
4,1082,17454,14793,17145,3566,5163,9550,2294




### Imports

We'll be using `scikit-learn`'s `ParameterGrid` for grid search, `self_driving_lab_demo`'s built-in
`get_random_inputs` for random search, and `ax-platform`'s Gaussian Process Expected
Improvement (GPEI) model for Bayesian
optimization. To help with defining the grid search space, we will also use the
`bounds` and `parameters` class property of `SelfDrivingLabDemo` for convenience.

In [9]:
import numpy as np
from tqdm import trange, tqdm
from sklearn.model_selection import ParameterGrid
from ax import optimize

In [10]:
sdls[0].bounds

{'brightness': [0.0, 0.35], 'R': [0, 255], 'G': [0, 255], 'B': [0, 255]}

In [11]:
sdls[0].parameters

[{'name': 'brightness', 'type': 'range', 'bounds': [0.0, 0.35]},
 {'name': 'R', 'type': 'range', 'bounds': [0, 255]},
 {'name': 'G', 'type': 'range', 'bounds': [0, 255]},
 {'name': 'B', 'type': 'range', 'bounds': [0, 255]}]

### Grid Search

First, we need to define our parameter grid. We'll divide up the 4-dimensional parameter
space as evenly as possible (see `num_pts_per_dim` below).

In [12]:
param_grid = {}
num_pts_per_dim = round(num_iter ** (1 / len(sdl.bounds)))
for name, bnd in sdl.bounds.items():
    param_grid[name] = np.linspace(bnd[0], bnd[1], num=num_pts_per_dim)
    if isinstance(bnd[0], int):
        param_grid[name] = np.round(param_grid[name]).astype(int)
print(f"num_pts_per_dim: {num_pts_per_dim}")

num_pts_per_dim: 3


Notice that there are only 3 distinct values along each dimension.

In [13]:
param_grid

{'brightness': array([0.   , 0.175, 0.35 ]),
 'R': array([  0, 128, 255]),
 'G': array([  0, 128, 255]),
 'B': array([  0, 128, 255])}

After assembling the full grid, notice that the total number of points is $3^4 = 81$.

In [14]:
grid = list(ParameterGrid(param_grid))
print("grid:\n", grid[0:4], "...", grid[-1:])
print("\nNumber of grid points: ", len(grid))

grid:
 [{'B': 0, 'G': 0, 'R': 0, 'brightness': 0.0}, {'B': 0, 'G': 0, 'R': 0, 'brightness': 0.175}, {'B': 0, 'G': 0, 'R': 0, 'brightness': 0.35}, {'B': 0, 'G': 0, 'R': 128, 'brightness': 0.0}] ... [{'B': 255, 'G': 255, 'R': 255, 'brightness': 0.35}]

Number of grid points:  81


Now, we can start the actual search. The grid search locations are fixed
for each of the repeat optimization campaigns; however the observed sensor data will be
stochastic and the target spectrum is different for each repeat run. An alternative approach to setting a
fixed budget and varying the target solution would be to see how many iterations it takes to meet a criteria for the
objective function similar to [this post](https://towardsdatascience.com/grid-search-vs-random-search-vs-bayesian-optimization-2e68f57c3c46); however, a fixed budget seems more characteristic of a real chemistry
or materials optimization campaign due to limits on funding, time, and other resources:
(i.e. we'll search until we find what we're looking for, until we run out of
resources, or until we decide it's no longer worth the expense, whichever comes first).

In [15]:
grid_data = [
    [
        sdl.evaluate(pt["brightness"], pt["R"], pt["G"], pt["B"])
        for pt in grid
    ]
    for sdl in tqdm(sdls)
]

100%|██████████| 5/5 [05:39<00:00, 67.86s/it]


### Random Search

Now, let's perform random search as we did before in
[`2.0-random-search.ipynb`](2.0-random-search.ipynb), storing the inputs and outputs as we go.

In [16]:
%%time
random_inputs = []
random_data = []
for _ in tqdm(range(num_repeats)):
    ri = []
    od = []
    for i in range(num_iter):
        ri.append(sdl.get_random_inputs())
        od.append(sdl.evaluate(*ri[i]))
    random_inputs.append(ri)
    random_data.append(od)

100%|██████████| 5/5 [05:39<00:00, 67.87s/it]

CPU times: user 34.8 s, sys: 29.6 s, total: 1min 4s
Wall time: 5min 39s


### Bayesian Optimization

Ax may run out of memory for low-RAM RPi's such as RPi Zero 2. Even on RPi 4B/RPi
400, it will be slow* for more than about 50 iterations due to the lack of MKL
optimization (a feature of RPi compute hardware).

<p>
<sup>
*i.e. maybe slower than a demo/tutorial should be
</sup>
</p>

In [17]:
%%time
bo_results = []

for sdl in tqdm(sdls):
    def evaluation_function(parameters):
        data = sdl.evaluate(
            parameters["brightness"],
            parameters["R"],
            parameters["G"],
            parameters["B"],
        )
        return data["mae"]

    bo_results.append(optimize(
        parameters=sdl.parameters,
        evaluation_function=evaluation_function,
        minimize=True,
        total_trials = num_iter,
    ))

best_parameters, values, experiment, model = zip(*bo_results)

  0%|          | 0/5 [00:00<?, ?it/s][INFO 08-20 17:25:12] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter brightness. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 08-20 17:25:12] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter R. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 08-20 17:25:12] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter G. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 08-20 17:25:12] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter B. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 's

CPU times: user 54min 27s, sys: 2min 20s, total: 56min 47s
Wall time: 36min 35s


### Analysis

Now that we've run our three optimizations, let's compare the performance in tabular
form and visually.

### Preparing the data

In [32]:
grid_mae = [[g["mae"] for g in gd] for gd in grid_data]
random_mae = [[r["mae"] for r in rd] for rd in random_data]
bayesian_mae = [exp.fetch_data().df["mean"].tolist() for exp in experiment]

In [33]:
mae = np.array([grid_mae, random_mae, bayesian_mae])
mae.shape

(3, 5, 81)

### Tabular

In [34]:
avg_mae = np.mean(np.minimum.accumulate(mae, axis=2), axis=1)
std_mae = np.std(avg_mae, axis=1)
avg_mae.shape

(3, 81)

In [35]:
np.mean(random_mae)

2818.2574074074073

In [36]:
best_avg_mae = np.min(avg_mae, axis=1)
best_avg_mae

array([1364.125,  707.9  ,  138.625])

### Best Objective vs. Iteration

In [37]:
names = ["grid", "random", "bayesian"]
df = pd.DataFrame({
    **{f"{n}_mae": m for n, m in zip(names, avg_mae)},
    **{f"{n}_std": s for n, s in zip(names, std_mae)},
})


In [38]:
mae_df = pd.melt(df.reset_index(), id_vars=["index"], value_vars = ["grid_mae", "random_mae", "bayesian_mae"], var_name="method", value_name="mae")

std_df = pd.melt(df.reset_index(), id_vars=["index"], value_vars = ["grid_std", "random_std", "bayesian_std"], var_name="method", value_name="std")

mae_df.loc[:, "method"] = mae_df.loc[:, "method"].apply(lambda x: x.replace("_mae", ""))
std_df.loc[:, "method"] = std_df.loc[:, "method"].apply(lambda x: x.replace("_std", ""))

In [39]:
results_df = mae_df.merge(std_df, on=["method", "index"]).rename(columns=dict(index="iteration"))
results_df

,iteration,method,mae,std
0,0,grid,4811.075,906.654837
1,1,grid,4810.525,906.654837
2,2,grid,4810.250,906.654837
3,3,grid,4809.875,906.654837
4,4,grid,4012.850,906.654837
...,...,...,...,...
238,76,bayesian,138.625,878.744510
239,77,bayesian,138.625,878.744510
240,78,bayesian,138.625,878.744510
241,79,bayesian,138.625,878.744510


### Visualization
As we might expect, Bayesian optimization outperforms random search while grid and
random search are on par with each other.

In [51]:
# import plotly.express as px
from self_driving_lab_demo.utils.plotting import line

fig = line(
    data_frame=results_df,
    x="iteration",
    y="mae",
    error_y="std",
    error_y_mode="band",
    color="method",
)
max_y = (results_df["mae"] + results_df["std"]).max()
fig.update_yaxes(range=[0.0, max_y*1.02])
fig

#### Example Output

![bayes-opt](bayesian-optimization.png)

## Repeat Measurement Stochasticity

In [42]:
sdls[0].get_target_inputs()

(0.33460059837014133, 53, 211, 38)

In [43]:
sdls[0].target_data

(725, 3680, 7084, 21221, 3924, 3668, 6416, 1724)

In [44]:
check_data = []
from time import sleep
for _ in range(10):
    check_data.append(sdls[0].evaluate(*sdls[0].get_target_inputs()))

In [45]:
pd.DataFrame(check_data).agg([np.mean, np.std]).T

,mean,std
ch415_violet,539.000000,0.471405
ch445_indigo,3498.500000,4.222953
ch480_blue,6869.900000,2.469818
ch515_cyan,20969.400000,16.466802
ch560_green,3604.100000,2.378141
ch615_yellow,3244.800000,12.072006
ch670_orange,5844.100000,2.424413
ch720_red,823.600000,3.806427
mae,381.075000,3.752407
rmse,446.694532,2.234845


## Code Graveyard

In [31]:
# for _ in trange(num_repeats):
#     for params in tqdm(grid):
#         sdl.evaluate(*params)

# pd.concat((mae_df, std_df), axis=1, join="inner")

# best_parameters = []
# values = []
# experiment = []
# model = []

    # best_parameters.append(bp)
    # values.append(v)
    # experiment.append(exp)
    # model.append(m)